This Jupyter Notebook is used to transform the output of xCropProtection into a time series map used for visualization. The output of this code is 2 html maps which show product applications to fields over time. These html files will be saved in user-defined file paths. The first 4 cells specify user-defined values; please review these cells before running the code.

- Map 1 visualizes the field applications based on product type (Fungicide, Insecticide, Herbicide, or other). Add products and their types to the product type.
- Map 2 visualizes the field applications based on produt name and assigns each product a unique random color.

Last update: November 6, 2024

Change file paths

In [ ]:
# Path to xCP output file (.dat file)
data_store_path = r'C:\path\to\arr.dat'

# Path to input shape file (used to display all fields on a map)
input_shp_file_path = r'C:\path\to\LULC.shp'

# Paths to write output html maps
output_map_html_path = r'D:\path\to\xCP_movie_product_type.html'
output_map_html_2_path =r'D:\path\to\xCP_movie_product_name.html'

# Table that specifies product names and types
product_table = r'C:\path\to\ProductTypes.csv'

Set the year to visualize

In [ ]:
map_year = "2021"

Set this value to True if output maps should show all field outlines

In [ ]:
show_field_outlines = True

Set the default zoom level of the maps

In [ ]:
zoom_level = 12

In [ ]:
import pandas
import geopandas
import h5py
import folium
from folium.plugins import TimestampedGeoJson
from IPython.display import display, HTML
from IPython.core.display import HTML
import datetime
import csv
from random import random
import colorsys

In [ ]:
# Check that all subgroups are h5py datasets
def checkInstance(datasets):
    for dataset in datasets:
        if not isinstance(dataset, h5py.Dataset):
            return False
    return True

# Try to access the hdf5 file
try:
    arr_file = h5py.File(data_store_path, 'r')
except FileNotFoundError:
    print("The file", data_store_path, "could not be accessed")

dataset = arr_file['xCropProtection']
landscape_dataset = arr_file['LandscapeScenario']

# Get data for subgroups
application_dates_subgroup = dataset['ApplicationDates']
application_rates_subgroup = dataset['ApplicationRates']
application_areas = dataset['AppliedAreas']
applied_features = dataset['AppliedFields']
application_PPP = dataset['AppliedPPP']
xcrop_file_path = dataset['xCropProtectionFilePath']
drift_reduction = dataset['TechnologyDriftReductions']
epsg = landscape_dataset['EPSG']

# Check that subgroups are h5py datasets
if not checkInstance([application_dates_subgroup, application_rates_subgroup, application_PPP, xcrop_file_path, drift_reduction]):
    print("Error retrieving subgroup data.")
    quit

In [ ]:
# Access data in each of the subgroups
application_dates_data = application_dates_subgroup[:]
application_rates_data = application_rates_subgroup[:]
applied_features_data = applied_features[:]
application_areas_data = application_areas[:]
application_PPP_data = application_PPP[:]
epsg_data = epsg[()]

# Construct dates from ordinals
application_dates = [datetime.date.fromordinal(x) for x in application_dates_data]

# Prepare data for a DataFrame
field_info_set = {'featureID': applied_features_data, 
                  'appliedPPP': [x.decode() for x in application_PPP_data], #Convert bytes to string
                  'appRate': application_rates_data, 
                  'appDate': pandas.to_datetime(application_dates), 
                  'appAreas': application_areas}
# Convert field_info_set dictionary to DataFrame
field_info_df = pandas.DataFrame(field_info_set)
field_info_df['appDate'] = field_info_df['appDate'].dt.strftime("%Y-%m-%d")
# Filter by selected year
field_info_df = field_info_df[field_info_df["appDate"].str.startswith(map_year)]

# Convert application area arrays to bytes for geometry creation
app_areas_bytes = [x.tobytes() for x in field_info_set['appAreas']]

# Create geodataframe and project geometry
geo_df = geopandas.GeoDataFrame(
    field_info_df,
    geometry=geopandas.GeoSeries.from_wkb(app_areas_bytes),
    crs="EPSG:" + str(epsg_data)
).to_crs(crs="EPSG:4258")
geo_df["field_idx"] = geo_df.reset_index().index
geo_df["date"] = field_info_df['appDate']
geo_df = geo_df.sort_values(by=["date"])
geo_df.drop(columns=['appAreas'], inplace=True)

# Get the center point so the map is centered on the correct area
df_centroid = geo_df.dissolve().centroid

Read shapefile for background field outlines

In [ ]:
input_shp_df = geopandas.read_file(input_shp_file_path)

Read product information (If you see "ï»¿" in the first entry, save the csv without UTF 8 (just csv))

In [ ]:
# product_dict
# keys: product names
# values: product type (ex: herbicide)
with open(product_table, mode='r') as prod_file:
    reader = csv.reader(prod_file)
    # Create a dictionary with product names as the keys and product types as the values
    product_dict = {}
    for row in reader:
        if row[0].startswith('"') and row[0].endswith('"'):
            row[0] = row[0][1:-1]
        product_dict[row[0]] = row[1]

unique_products = geo_df['appliedPPP'].unique()

# product_colors_dict
# keys: product names
# values: color values as hex (ex: #000000) 
product_colors_dict = {}
for product in unique_products:
    # Generate bright colors
    h,s,l = random(), 0.5 + random()/2.0, 0.4 + random()/5.0
    r,g,b = [int(256*i) for i in colorsys.hls_to_rgb(h,l,s)]
    product_colors_dict[product] = '#%02X%02X%02X' % (r, g, b)

Define map 1 colors

In [ ]:
# Change this list to edit product colors on the map
prod_type_colors = {'Fungicide': '#4477AA', 'Insecticide': '#CC3311', 'Herbicide': '#CCBB44', 'Other': '#306B34'}

def get_color(prod_name):
    # Get the product's type from product_dict, then get that product type's color
    return prod_type_colors.get(product_dict.get(prod_name))

In [ ]:
swatch_dict = {}

for key in prod_type_colors:
    # Create color block for the legend
    color_swatch = f'<div style="background-color: {prod_type_colors[key]}; width: 10px; height: 10px; display: inline-block;"></div>'
    # Add swatch to the dictionary
    swatch_dict[key] = color_swatch

product_swatch_dict = {}


for key in product_colors_dict:
    # Create color block for the legend
    color_swatch = f'<div style="background-color: {product_colors_dict[key]}; width: 10px; height: 10px; display: inline-block;"></div>'
    # Only add a swatch to the legend if that product is used in this scenario
    if key in unique_products:
        # Add swatch to the dictionary
        product_swatch_dict[key] = color_swatch

Create map 1 features

In [ ]:
# Create features to add to the map
map_1_features = []

for _, row in geo_df.iterrows():
    feature = {
        'type': 'Feature',
        'geometry': row['geometry'].__geo_interface__,
        'properties': {
            'times': [row['date']],
            'popup': str(row['appliedPPP']).strip('\"') + ": " + str(round(row['appRate'], 2)) + " g/ha",
            'style': {
                'color': get_color(row['appliedPPP']),
                'fillOpacity': 1.0,
                'stroke': False
            }
        }
    }
    map_1_features.append(feature)

Create map 2 features

In [ ]:
map_2_features = []

for _, row in geo_df.iterrows():
    feature = {
        'type': 'Feature',
        'geometry': row['geometry'].__geo_interface__,
        'properties': {
            'times': [row['date']],
            'popup': str(row['appliedPPP']).strip('\"') + ": " + str(round(row['appRate'], 2)) + " g/ha",
            'style': {
                'color': product_colors_dict.get(row['appliedPPP']),
                'fillOpacity': 1.0,
                'stroke': False
            }
        }
    }
    map_2_features.append(feature)

Generate and display map 1

In [ ]:
# Set map lat/lon and zoom here
map_1 = folium.Map(location=[df_centroid.y.iloc[0], df_centroid.x.iloc[0]], zoom_start = zoom_level, tiles = "CartoDB Positron")

layer_1 = TimestampedGeoJson({'type': 'FeatureCollection', 'features': map_1_features}, period="P1D", date_options='YYYY-MM-DD', add_last_point=False,
                            duration="P1D", time_slider_drag_update=True, loop=False, transition_time=500)

layer_1.add_to(map_1)

if show_field_outlines:
    background_layer = folium.GeoJson(input_shp_df, style_function=lambda feature: {
        "color": "#D3D4D3",
        "fillOpacity": 0,
        "weight": 1,
    },)
    
    background_layer.add_to(map_1)

# Legend for fields colored based on applied PPP
legend_3 = '''
<div style="position: fixed; bottom: 50px; right: 10px; z-index:1000; background-color: white; padding: 2px; border: 1px solid grey; font-size: 12px;">
    <div style="background-color: white; color: black; padding: 2px; border: 1px solid grey;">Legend</div>
'''
# Add each PPP type and its corresponding color
for key in swatch_dict:
    legend_3 += '''<div style="background-color: white; color: black; padding: 2px; border: 1px solid grey;">{}: {}</div>'''.format(key, swatch_dict[key])

legend_3 += '''</div>
</div>
'''

# Add legend to map
map_1.get_root().html.add_child(folium.Element(legend_3))

folium.plugins.Fullscreen(
    position="topright",
    title="Fullscreen",
    title_cancel="Exit fullscreen",
    force_separate_button=True,
).add_to(map_1)

htmlmap_1 = HTML('<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; margin: 0 auto; border: 1px solid black"></iframe>'
           .format(map_1.get_root().render().replace('"', '&quot;'),650,650))
display(htmlmap_1)


Generate and display map 2

In [ ]:
# Set map lat/lon and zoom here
map_2 = folium.Map(location=[df_centroid.y.iloc[0], df_centroid.x.iloc[0]], zoom_start = zoom_level, tiles = "CartoDB Positron")

layer_2 = TimestampedGeoJson({'type': 'FeatureCollection', 'features': map_2_features}, period="P1D", date_options='YYYY-MM-DD', add_last_point=False,
                            duration="P1D", time_slider_drag_update=True, loop=False, transition_time=500)

layer_2.add_to(map_2)
if show_field_outlines:
    background_layer.add_to(map_2)

# Legend for fields colored based on applied PPP
map_2_legend = '''
<div style="position: fixed; bottom: 50px; right: 10px; z-index:1000; background-color: white; padding: 2px; border: 1px solid grey; font-size: 12px;">
    <div style="background-color: white; color: black; padding: 2px; border: 1px solid grey;">Legend</div>
'''
# Add each PPP type and its corresponding color
for key in product_swatch_dict:
    map_2_legend += '''<div style="background-color: white; color: black; padding: 2px; border: 1px solid grey;">{}: {}</div>'''.format(key.strip('\"'), product_swatch_dict[key])

map_2_legend += '''</div>
</div>
'''

# Add legend to map
map_2.get_root().html.add_child(folium.Element(map_2_legend))

folium.plugins.Fullscreen(
    position="topright",
    title="Fullscreen",
    title_cancel="Exit fullscreen",
    force_separate_button=True,
).add_to(map_2)

htmlmap_2 = HTML('<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; margin: 0 auto; border: 1px solid black"></iframe>'
           .format(map_2.get_root().render().replace('"', '&quot;'),650,650))

display(htmlmap_2)

Uncomment code to format both maps to be displayed on one screen

In [ ]:
htmlmap_2_maps = HTML('<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; margin: 0 auto; border: 1px solid black"></iframe>'
               '<iframe srcdoc="{}" style="float:right; width: {}px; height: {}px; display:inline-block; margin: 0 auto; border: 1px solid black"></iframe>'
            .format(map_1.get_root().render().replace('"', '&quot;'),650,650, 
                   map_2.get_root().render().replace('"', '&quot;'),650,650))

#display(htmlmap_2_maps)

In [ ]:
html_content = htmlmap_1.data
html_content_2 = htmlmap_2.data

# Write the HTML content to a file
with open(output_map_html_path, "w") as file:
    file.write(html_content)

with open(output_map_html_2_path, "w") as file:
    file.write(html_content_2)